In [327]:
def training_simulator(initial_ADOS_trained, initial_ADIR_trained, total_trainers, total_trainees, ados_in_training, adir_in_training, ados_trained, adir_trained, desired_ADOS_ADIR_ratio, ADOS_training_time, ADIR_training_time, ados_cost, adir_cost, annual_budget, training_months, forecast_horizon):

    import pandas as pd

    # Initialize trainers with type as 0 and time_left as 0
    trainers = [{"train_type": "none", "time_left": -1} for _ in range(total_trainers)]

    try_ados = True
    try_adir = True

    current_budget = 0
    wasted_budget = 0

    df = pd.DataFrame()

    # Simulation for intended forecast period
    for qmonth in range(1, forecast_horizon*4 + 1):

        # check if it's a new year: budget renewal if still within training years, zero budget otherwise
        budget_reset_month = False
        if qmonth % 48 == 1:
            wasted_budget = current_budget
            if qmonth < (training_months * 4):
                current_budget = annual_budget
                budget_reset_month = True
            else:
                current_budget = 0
        else:
            wasted_budget = 0

        # Figure out what to do for each trainer each quarter-month if there's money left
        for trainer in range(0, total_trainers):

            train_type = trainers[trainer]["train_type"]
            time_left = trainers[trainer]["time_left"]

            # if training done, decrease in-training count and increase trained doctor count
            # else if this trainer was switched off previously but new budget has come in, ready the trainer again
            # else reduce training time left
            if (time_left == 0):

                if train_type == "ADOS":
                    ados_in_training = ados_in_training - 1
                    ados_trained = ados_trained + 1
                else:
                    adir_in_training = adir_in_training - 1
                    adir_trained = adir_trained + 1
            
            elif (time_left == -1) and budget_reset_month:
                trainers[trainer]["train_type"] = "none"
                    
            else:
                trainers[trainer]["time_left"] = time_left - 1

            # if training is done or we're just starting out, find new training type if there are still potential trainees left
            if (time_left == 0) or (train_type == "none"):
                if try_ados or try_adir:
                    
                    # calculate ADOS/ADI-R ratios
                    if (ados_trained + adir_trained + ados_in_training + adir_in_training) > 0:
                        total_ratio = (ados_trained + ados_in_training) / (ados_trained + adir_trained + ados_in_training + adir_in_training)
                    else:
                        total_ratio = 0

                    if (ados_trained + adir_trained) > 0:
                        trained_ratio = (ados_trained) / (ados_trained + adir_trained)
                    else:
                        trained_ratio = 0

                    # set flags if out of budget
                    # otherwise, check for trainer/trainee availability
                    if (current_budget < min(ados_cost, adir_cost)):
                        trainers[trainer]["train_type"] = "no budget"
                        trainers[trainer]["time_left"] = -1
                        
                    else:
                        
                        # flag trainer as ready - important for ADI-R check within this block
                        trainers[trainer]["train_type"] = "no trainee"

                        # add ADOS trainee IFF not enough ADOS trained and we can still train more and we have budget
                        if (total_ratio < desired_ADOS_ADIR_ratio) and try_ados and (current_budget >= ados_cost):
                            if (ados_trained < total_trainees):
                                trainers[trainer]["train_type"] = "ADOS"
                                trainers[trainer]["time_left"] = ADOS_training_time
                                ados_in_training = ados_in_training + 1
                                current_budget = current_budget - ados_cost

                                # check if we are out of potential for ADOS training
                                if ados_trained + ados_in_training >= total_trainees:
                                    try_ados = False
                            else:
                                try_ados = False

                        # add ADI-R trainee if this trainer is not currently doing ADOS training
                        # and we shouldn't or can't train new ADOS doctors
                        # no need to check budget because budget checked again ADOS+ADIR in outer conditional
                        # and we would only get to this block if there is budget left but not for ADOS
                        if trainers[trainer]["train_type"] != "ADOS":
                            if (total_ratio >= desired_ADOS_ADIR_ratio) or not try_ados:
                                if (adir_trained < total_trainees):

                                    trainers[trainer]["train_type"] = "ADI-R"
                                    trainers[trainer]["time_left"] = ADIR_training_time
                                    adir_in_training = adir_in_training + 1
                                    current_budget = current_budget - adir_cost

                                    # check if we are out of potential for ADI-R training
                                    if adir_trained + adir_in_training >= total_trainees:
                                        try_adir = False
                            else:
                                try_adir = False

                else:

                    # set time left and train_type to "no potential" values
                    trainers[trainer]["time_left"] = -1
                    trainers[trainer]["train_type"] = "all trained"

        # add this month's numbers to the df
        new_data = {'qmonth': qmonth, 'ADOS in training': ados_in_training, 'ADOS trained': ados_trained, 'ADI-R in training': adir_in_training, 'ADI-R trained': adir_trained, "total ratio": total_ratio, "trained ratio": trained_ratio, 'budget left': current_budget, 'wasted budget': wasted_budget}
        df = df.append(new_data, ignore_index=True)

    # Final results

    df_month = df.iloc[::4]
    df_month['month'] = (df_month['qmonth'] - 1) / 4 + 1
    df_month['month'] = df_month['month'].astype(int)
    df_month = df_month.drop(columns=['qmonth'])
    df_month.reset_index(drop=True, inplace=True)

    df_month['new ADOS'] = df_month['ADOS trained'] - df_month['ADOS trained'].shift(1, fill_value=0)
    df_month['new ADI-R'] = df_month['ADI-R trained'] - df_month['ADI-R trained'].shift(1, fill_value=0)
    
    df_month = df_month[['month', 'ADOS in training', 'ADI-R in training', 'ADOS trained', 'ADI-R trained', 'new ADOS', 'new ADI-R', 'total ratio', 'trained ratio', 'budget left', 'wasted budget']]
    print(df_month)

    # Create veterancy columns
    for i in range(7):
        df_month[f'ADOS {i}'] = 0

    for i in range(7):
        df_month[f'ADI-R {i}'] = 0

    # Set initial values in the first row
    df_month.loc[0, 'ADOS 6'] = initial_ADOS_trained
    df_month.loc[0, 'ADI-R 6'] = initial_ADIR_trained

    # Iterate through rows and perform the shifting
    # 6-month veterans are replicated in the new month
    # all other docs are promoted 1 month of veterancy
    for i in range(1, len(df_month)):
        df_month.loc[i, 'ADOS 6'] = df_month.loc[i - 1, 'ADOS 5'] + df_month.loc[i - 1, 'ADOS 6']
        df_month.loc[i, 'ADOS 5'] = df_month.loc[i - 1, 'ADOS 4']
        df_month.loc[i, 'ADOS 4'] = df_month.loc[i - 1, 'ADOS 3']
        df_month.loc[i, 'ADOS 3'] = df_month.loc[i - 1, 'ADOS 2']
        df_month.loc[i, 'ADOS 2'] = df_month.loc[i - 1, 'ADOS 1']
        df_month.loc[i, 'ADOS 1'] = df_month.loc[i - 1, 'ADOS 0']
        df_month.loc[i, 'ADOS 0'] = df_month.loc[i - 1, 'new ADOS']
        df_month.loc[i, 'ADI-R 6'] = df_month.loc[i - 1, 'ADI-R 5'] + df_month.loc[i - 1, 'ADI-R 6']
        df_month.loc[i, 'ADI-R 5'] = df_month.loc[i - 1, 'ADI-R 4']
        df_month.loc[i, 'ADI-R 4'] = df_month.loc[i - 1, 'ADI-R 3']
        df_month.loc[i, 'ADI-R 3'] = df_month.loc[i - 1, 'ADI-R 2']
        df_month.loc[i, 'ADI-R 2'] = df_month.loc[i - 1, 'ADI-R 1']
        df_month.loc[i, 'ADI-R 1'] = df_month.loc[i - 1, 'ADI-R 0']
        df_month.loc[i, 'ADI-R 0'] = df_month.loc[i - 1, 'new ADI-R']

    return df_month

In [328]:
# Set initial values

# notes: to force variability across budgets, what we want is to make sure the training budget is used up quickly
# we can do this by reducing training time, increasing trainer pool, or increasing training cost
# this is hard-capped by the trainee pool size, so increase that to prevent capping out
# this results in the lower budget settings churning out fewer NHS doctors, but being able to spend more on locums in year 1

# to force variability across years, what we want is to change how many years we get to refresh our training budget
# rather than rework my code, I'll just run this multiple times - once for each training_duration value

initial_ADOS_trained = 12 #12
initial_ADIR_trained = 8 #8

total_trainers = 30 #20
total_trainees = 5000 #100
ados_in_training = 0
adir_in_training = 0
ados_trained = 0
adir_trained = 0

# training time in quarter-months
ADOS_training_time = 20 #20
ADIR_training_time = 4 #4

# training costs and budget
ados_cost = 10000 #5000
adir_cost = 2000 #1000
annual_budget = 400000 #350000
budget_step = 100000 #50000 # gap to use between budgets to consider

# assessment costs by NHS doctors (separate ADOS and ADI-R) and locums (both assessments together)
NHS_ADOS_cost_per_patient = 48 # 480
NHS_ADIR_cost_per_patient = 9.5 # 95
locum_cost_per_patient = 1000 #1000

# assessment times per patient
ADOS_time = 3 #1
ADIR_time = 4.5 #1.5
desired_ADOS_ADIR_ratio = ADIR_time / (ADIR_time + ADOS_time)

training_duration = 24 # we are only calcuating outcomes for 12 months' spending
forecast_horizon = 60 # project for 5 years

# veterancy numbers
base_weekly_hours = 3 #7.5
veterancy_factor = 2 #10
veteran_month = 6 # at how many months is a doctor a veteran?

waitlist_start = 7698 #7698
daily_new_patients = 12 #6

In [329]:
import pandas as pd

# Suppress all warnings
import warnings
warnings.filterwarnings("ignore")

# what I want to do now is to generate a table of total patient clearance per year
# for training spend of 0 - 350k in year 1 (super large tables if we want to check different spending years)
# with forecast horizon up to 5 years from the start year

# Weekly hours available per NHS doctor = 8 + 10*veterancy
# where veterancy increases linearly from 0 at start to 1 at 6 months
# hours available = 8 + 10/5 * (months -1 capped at 5)

# each patient is only cleared after both ADOS and ADI-R

# so first calculate the monthly contributions of all available NHS doctors (with veterancy considered)
# then add these up for annual contribution
# then add in locum contributions based on annual budget left over for them

# for each budget from 0 - max
# for each month of forecast up to 5 years
# calculate patients left in system after clearing and gaining

budget_list = list(range(0, annual_budget + 1, budget_step))
df = pd.DataFrame()

for budget in budget_list:
    
    NHS_ADOS_hours = 0
    NHS_ADIR_hours = 0
    NHS_ADOS_cost = 0
    NHS_ADIR_cost = 0
    max_NHS_patients_seen = 0
    
    training_budget = budget
    training_outcomes = training_simulator(initial_ADOS_trained, initial_ADIR_trained, total_trainers, total_trainees, ados_in_training, adir_in_training, ados_trained, adir_trained, desired_ADOS_ADIR_ratio, ADOS_training_time, ADIR_training_time, ados_cost, adir_cost, training_budget, training_duration, forecast_horizon)

    # debug
    #print(budget)
    #print(training_outcomes[['month','ADOS trained', 'ADI-R trained', 'budget left', 'ADOS 6', 'ADI-R 6']].iloc[[*range(0, 13), *range(56, 59)]])
    #print(training_outcomes[['month','ADOS trained', 'ADI-R trained', 'budget left', 'ADOS 6', 'ADI-R 6']])
    
    total_patients_seen = []
    
    for month in range(0, forecast_horizon):
        
        for veterancy in range (0,7):
            hours_per_doctor_per_week = base_weekly_hours + (veterancy_factor/(veteran_month - 1)*(veterancy - 1))
            
            ADOS_key = f'ADOS {veterancy}'
            ADOS_doctors = training_outcomes[ADOS_key][month]
            NHS_ADOS_hours = NHS_ADOS_hours + (hours_per_doctor_per_week*ADOS_doctors*4)
            
            ADIR_key = f'ADI-R {veterancy}'
            ADIR_doctors = training_outcomes[ADIR_key][month]
            NHS_ADIR_hours = NHS_ADIR_hours + (hours_per_doctor_per_week*ADIR_doctors*4)
        
        # across all veterancy levels for this month, we have
        NHS_ADOS_patients = NHS_ADOS_hours/ADOS_time
        NHS_ADIR_patients = NHS_ADIR_hours/ADIR_time
        
        # we assume that we only train doctors in either ADOS or ADI-R early on, but eventually they will be double-trained
        # double-trained doctors don't have double the contribution time and have to choose between the assessments
        # so what we need to do is to count up the doubles
        # and then reduce the two patient counts above
        # assume that initial trained docs are not doubled up
        docs_on_hand = ADOS_doctors + ADIR_doctors
        if docs_on_hand > (total_trainees + initial_ADOS_trained + initial_ADIR_trained):
            for i in range (0, docs_on_hand):
                if NHS_ADOS_patients > NHS_ADIR_patients:
                    NHS_ADOS_patients -= 1
                else:
                    NHS_ADIR_patients -= 1
                    # doesn't matter if we always decrease ADIR first since the next step would balane the numbers again
                    # and in the final equation the lower of the two NHS counters is used
        
        # we also add this month's patients seen to the counter (this counter is reset at end of year)
        max_NHS_patients_seen = max_NHS_patients_seen + int(min(NHS_ADOS_patients, NHS_ADIR_patients))

        # at the end of each year, tally up numbers
        if month % 12 == 11:
            
            this_year = (month + 1) / 12
            NHS_ADOS_cost = NHS_ADOS_cost_per_patient * max_NHS_patients_seen
            NHS_ADIR_cost = NHS_ADIR_cost_per_patient * max_NHS_patients_seen
            total_NHS_cost = NHS_ADOS_cost + NHS_ADIR_cost
            
            # check if we are over budget and constrain if so
            if total_NHS_cost > annual_budget:
                print('over budget!', total_NHS_cost, annual_budget)
                max_NHS_patients_seen = int(annual_budget / (NHS_ADOS_cost_per_patient + NHS_ADIR_cost_per_patient))
                total_NHS_cost = max_NHS_patients_seen * (NHS_ADOS_cost_per_patient + NHS_ADIR_cost_per_patient)
            
            locum_spend = max(annual_budget - budget - total_NHS_cost, 0)
            locum_patients = int(locum_spend / locum_cost_per_patient)

            # add this years's numbers to the list
            total_patients_seen.append(max_NHS_patients_seen + locum_patients)
            
            # reset lists for next year
            NHS_ADOS_cost = 0
            NHS_ADIR_cost = 0
            max_NHS_patients_seen = 0
            
        # reset NHS hours for the next month
        NHS_ADOS_hours = 0
        NHS_ADIR_hours = 0

    # collate data for this budget and add to df
    new_data = {'training budget': budget}
    for year in range(len(total_patients_seen)):
        key = f'patients year {year + 1}'
        new_data[key] = total_patients_seen[year]
    df = df.append(new_data, ignore_index=True)
    print('budget', budget, 'done....')

print('Number of patients cleared per year for different training budgets')

df

    month  ADOS in training  ADI-R in training  ADOS trained  ADI-R trained  \
0       1                 0                  0             0              0   
1       2                 0                  0             0              0   
2       3                 0                  0             0              0   
3       4                 0                  0             0              0   
4       5                 0                  0             0              0   
5       6                 0                  0             0              0   
6       7                 0                  0             0              0   
7       8                 0                  0             0              0   
8       9                 0                  0             0              0   
9      10                 0                  0             0              0   
10     11                 0                  0             0              0   
11     12                 0                  0      

    month  ADOS in training  ADI-R in training  ADOS trained  ADI-R trained  \
0       1               8.0                6.0           0.0            0.0   
1       2               8.0                6.0           0.0            0.0   
2       3               8.0              -18.0           0.0           24.0   
3       4               8.0              -42.0           0.0           48.0   
4       5               8.0              -66.0           0.0           72.0   
5       6               8.0              -90.0           0.0           96.0   
6       7               0.0             -138.0           8.0          144.0   
7       8               0.0             -194.0           8.0          200.0   
8       9               0.0             -250.0           8.0          256.0   
9      10               0.0             -306.0           8.0          312.0   
10     11               0.0             -362.0           8.0          368.0   
11     12               0.0             -418.0      

    month  ADOS in training  ADI-R in training  ADOS trained  ADI-R trained  \
0       1              17.0               12.0           0.0            0.0   
1       2              17.0               12.0           0.0            0.0   
2       3              17.0              -36.0           0.0           48.0   
3       4              17.0              -84.0           0.0           96.0   
4       5              17.0             -132.0           0.0          144.0   
5       6              17.0             -180.0           0.0          192.0   
6       7               0.0             -279.0          17.0          291.0   
7       8               0.0             -395.0          17.0          407.0   
8       9               0.0             -511.0          17.0          523.0   
9      10               0.0             -627.0          17.0          639.0   
10     11               0.0             -743.0          17.0          755.0   
11     12               0.0             -859.0      

    month  ADOS in training  ADI-R in training  ADOS trained  ADI-R trained  \
0       1              18.0               12.0           0.0            0.0   
1       2              18.0               12.0           0.0            0.0   
2       3              25.0                5.0           0.0           12.0   
3       4              26.0               -5.0           0.0           23.0   
4       5              26.0              -19.0           0.0           37.0   
5       6              26.0              -35.0           0.0           53.0   
6       7               8.0             -105.0          18.0          123.0   
7       8               1.0             -207.0          25.0          225.0   
8       9               0.0             -324.0          26.0          342.0   
9      10               0.0             -444.0          26.0          462.0   
10     11               0.0             -564.0          26.0          582.0   
11     12               0.0             -684.0      

    month  ADOS in training  ADI-R in training  ADOS trained  ADI-R trained  \
0       1              18.0               12.0           0.0            0.0   
1       2              18.0               12.0           0.0            0.0   
2       3              25.0                5.0           0.0           12.0   
3       4              28.0                2.0           0.0           17.0   
4       5              29.0                1.0           0.0           19.0   
5       6              30.0                0.0           0.0           20.0   
6       7              17.0              -23.0          18.0           47.0   
7       8              10.0              -85.0          25.0          109.0   
8       9               7.0             -168.0          28.0          192.0   
9      10               6.0             -260.0          29.0          284.0   
10     11               6.0             -356.0          29.0          380.0   
11     12               0.0             -465.0      

,training budget,patients year 1,patients year 2,patients year 3,patients year 4,patients year 5
0,0,795,795,795,795,795
1,100000,1246,1991,2478,2482,2482
2,200000,1339,2737,3663,3672,3672
3,300000,1367,3550,5089,5112,5112
4,400000,1376,4230,6457,6552,6552


In [330]:
# create table of cumulative clears
cum_df = df.copy()

num_columns = int(forecast_horizon/12) + 1

# Iterate through columns and perform the operation
for i in range(1, num_columns - 1):
    cum_df.iloc[:, i + 1] += cum_df.iloc[:, i]

print('cumulative clears')
cum_df

cumulative clears


,training budget,patients year 1,patients year 2,patients year 3,patients year 4,patients year 5
0,0,795,1590,2385,3180,3975
1,100000,1246,3237,5715,8197,10679
2,200000,1339,4076,7739,11411,15083
3,300000,1367,4917,10006,15118,20230
4,400000,1376,5606,12063,18615,25167


In [331]:
# create table of patients left in waitlist

df_left = cum_df.copy()

# Each cell except first column starts with waitlist_start value - patient clear numbers
df_left.iloc[:, 1:] = waitlist_start - df_left.iloc[:, 1:]

# Add new patients (6 daily) to cells
for writecol in range (1, (int(forecast_horizon/12) + 1)):
    df_left.iloc[:, writecol] = df_left.iloc[:, writecol] + (daily_new_patients * 365 * writecol)

# Apply a non-negative constraint to the entire DataFrame
df_left[df_left < 0] = 0

print('Number of patients left in waitlist for different training budgets')

csv_name = f'budget_{int(annual_budget/1000)}k_train_{int(training_duration/12)}yr.csv'
df_left.to_csv(csv_name, index=False)
df_left

Number of patients left in waitlist for different training budgets


,training budget,patients year 1,patients year 2,patients year 3,patients year 4,patients year 5
0,0,11283,14868,18453,22038,25623
1,100000,10832,13221,15123,17021,18919
2,200000,10739,12382,13099,13807,14515
3,300000,10711,11541,10832,10100,9368
4,400000,10702,10852,8775,6603,4431
